## Physics Informed Neural Networks to Approximate Solution of PDEs

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
import torch
from torch.utils.data import DataLoader, TensorDataset

# Adapt this import to your specific directory tree.
from Common import NeuralNet, MultiVariatePoly
import time
torch.autograd.set_detect_anomaly(True)
torch.manual_seed(128)

In [2]:
input_data = torch.from_numpy(np.load("input.npy")).type(torch.float32)
input_mu = torch.from_numpy(np.load("input_mu.npy")).type(torch.float32)
output_data = torch.from_numpy(np.load("output.npy")).type(torch.float32)
print(input_data.shape)
print(input_mu.shape)
print(output_data.shape)



torch.Size([32, 100, 100, 3])
torch.Size([32, 2])
torch.Size([32, 100, 100])


In [3]:
total_input = []
for i in range (input_data.shape[0]):
    for j in range (input_data.shape[1]):
        for k in range (input_data.shape[1]):
            total_input.append(np.append(input_data[i][j][k][:-1].flatten().numpy(), input_mu[i].flatten().numpy() ))
print(total_input[0])
print(total_input[0].shape)
print(type(total_input[0]))

[-1.        -1.        -0.8806173  0.9568935]
(4,)
<class 'numpy.ndarray'>


In [7]:
total_input = np.vstack(total_input)
print(type(total_input))
# 

<class 'numpy.ndarray'>


In [8]:
total_input = torch.from_numpy(total_input)


In [12]:
print((total_input[0]))
print(total_output[0])
total_output = torch.flatten(output_data)
print(total_output.shape)
print(total_input.shape)

tensor([-1.0000, -1.0000, -0.8806,  0.9569])
tensor(1.7406e-19)
torch.Size([320000])
torch.Size([320000, 4])


In [13]:
ip_shape = 4
op_shape = 1
model = NeuralNet(input_dimension=ip_shape,
                            output_dimension=op_shape,
                            n_hidden_layers=4,
                            neurons=20,
                            regularization_param=0.,
                            regularization_exp=2.,
                            retrain_seed=42)

In [17]:
optimizer_ADAM = optim.Adam(model.parameters(),
                            lr=float(0.005))

ratio_train = 0.8                            
n_train = int(np.round(len(total_input) * ratio_train))

train_input = total_input[:n_train]
train_output = total_output[:n_train]
test_input = total_input[n_train:]
test_output = total_output[n_train:]

batch_size = 16

training_set = DataLoader(TensorDataset(train_input, train_output), batch_size=batch_size, shuffle=False)
testing_set = DataLoader(TensorDataset(test_input, test_output), batch_size=batch_size, shuffle=False)



print(train_input.shape)
print(train_output.shape)
print(test_output.shape)
print(test_input.shape)

torch.Size([256000, 4])
torch.Size([256000])
torch.Size([64000])
torch.Size([64000, 4])


In [19]:
def fit(model, training_set, num_epochs, optimizer, p, verbose=True):
    history = list()

    # Loop over epochs
    for epoch in range(num_epochs):
        if verbose: print("################################ ", epoch, " ################################")

        running_loss = list([0])

        # Loop over batches
        for j, (x_train_, u_train_) in enumerate(training_set):
            def closure():
                # zero the parameter gradients
                optimizer.zero_grad()
                # forward + backward + optimize
                u_pred_ = model(x_train_)
                # Item 1. below
                loss = torch.mean((u_pred_.reshape(-1, ) - u_train_.reshape(-1, )) ** p) + model.regularization()
                # Item 2. below
                loss.backward()
                # Compute average training loss over batches for the current epoch
                running_loss[0] += loss.item()
                return loss

            # Item 3. below
            optimizer.step(closure=closure)

        if verbose: print('Loss: ', (running_loss[0] / len(training_set)))
        history.append(running_loss[0])

    return history
epochs = 3
hist = fit(model= model, training_set=training_set, num_epochs=epochs, optimizer=optimizer_ADAM, p=2, verbose=True)

################################  0  ################################
Loss:  0.002641145908329501
################################  1  ################################
Loss:  1.774182580819683e-05
################################  2  ################################
Loss:  0.00040139439059229024
